<a href="https://colab.research.google.com/github/jstralko/lnl-machine-learning/blob/master/ch6-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Here’s the code that loads MNIST images into X:

In [0]:
import numpy as np
import gzip
import struct
import requests
import io

def fetch_file(filename):
  r = requests.get('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/tensorflow/MNIST_data/' + filename)
  return io.BytesIO(r.content)

def load_images(filename):
  
  # Open and unzip the file of images:
  with gzip.open(fetch_file(filename), 'rb') as f:
  
    # Read the header information into a bunch of variables:
    _ignored, n_images, columns, rows = struct.unpack('>IIII', f.read(16))
    # Read all the pixels into a NumPy array:
    all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
  
    # Reshape the pixels into a matrix where each line is an image:
    return all_pixels.reshape(n_images, columns * rows)

def prepend_bias(X):
  # Insert a column of 1s in the position 0 of X.
  # (“axis=1” stands for: “insert a column, not a row”)
  return np.insert(X, 0, 1, axis=1)
  
# 60000 images, each 785 elements (1 bias + 28 * 28 pixels)
X_train = prepend_bias(load_images("train-images-idx3-ubyte.gz"))
# 10000 images, each 785 elements, with the same structure as X_train
X_test = prepend_bias(load_images("t10k-images-idx3-ubyte.gz"))

Should have training and test data. See X_train, X_test

```
(60000, 785)
(10000, 785)
```

In [36]:
print(X_train.shape)
print(X_test.shape)

#See the bias loaded
for i in range (0,5):
  print(X_train[i][0])

(60000, 785)
(10000, 785)
1
1
1
1
1


This code loads and prepares MNIST’s labels:

In [0]:
def load_labels(filename):
  # Open and unzip the file of images:
  with gzip.open(fetch_file(filename), 'rb') as f:
    # Skip the header bytes:
    f.read(8)
    
    # Read all the labels into a list:
    all_labels = f.read()
    
    # Reshape the list of labels into a one-column matrix:
    return np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)

def encode_fives(Y):
  encoded_Y = np.zeros_like(Y)
  n_labels = Y.shape[0]
  for i in range(n_labels):
    if Y[i] == 5: 
      encoded_Y[i][0] = 1
  return encoded_Y

# 60K labels, each with value 1 if the digit is a five, and 0 otherwise
Y_label = load_labels("train-labels-idx1-ubyte.gz")
Y_train = encode_fives(Y_label)
# 10000 labels, with the same encoding as Y_train
Y_test_label = load_labels("t10k-labels-idx1-ubyte.gz")
Y_test = encode_fives(Y_test_label)


Should be this:

```
(60000, 1)
(10000, 1)
```

In [42]:
print(Y_train.shape)
print(Y_test.shape)

#How many 5s    
print(len(list(filter(lambda x: x == 0 , Y_train))))
#How many not 5s
print(len(list(filter(lambda x: x == 1 , Y_train))))

(60000, 1)
(10000, 1)
54579
5421


The fun part. First add the Logistic Regress from chapter 5:

In [0]:
#Chapter 5: page 66

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Forward propagation: moving data
# through the system
def forward(X, w):
    weighted_sum = np.matmul(X, w)
    return sigmoid(weighted_sum)

#Formerly predict()
def classify(X, w):
    return np.round(forward(X, w))

def loss(X, Y, w):
    y_hat = forward(X, w)
    first_term = Y * np.log(y_hat)
    second_term = (1 - Y) * np.log(1 - y_hat)
    return -np.average(first_term + second_term)

def gradient(X, Y, w):
    return np.matmul(X.T, (forward(X, w) - Y)) / X.shape[0]

def train(X, Y, iterations, lr):
    w = np.zeros((X.shape[1], 1))
    for i in range(iterations):
        print("Iteration %4d => Loss: %.20f" % (i, loss(X, Y, w)))
        w -= gradient(X, Y, w) * lr
    return w

def test(X, Y, w, number=5):
    total_examples = X.shape[0]
    correct_results = np.sum(classify(X, w) == Y)
    success_percent = correct_results * 100 / total_examples
    
    print("[%s] Success %d/%d: (%3.2f)" % 
          (number, correct_results,
           total_examples,
           correct_results * 100 / total_examples)
         )


Add train and test the data!

In [51]:
# To bypass all the loading and preprocessing of the data.
# mnist library handles all this for you.
# import mnist as data
# w = train(data.X_train, data.Y_train, iterations=100, lr=1e-5)
# test(data.X_test, data.Y_test, w)

#Learning Rates
#lr=.0001 - Good
#lr=.001 - Bad
w = train(X_train, Y_train, iterations=10, lr=1e-5)
test(X_test, Y_test, w)

Iteration    0 => Loss: 0.69314718055994528623
Iteration    1 => Loss: 0.80042530259490185518
Iteration    2 => Loss: 0.60370180008019180828
Iteration    3 => Loss: 0.41561114405207100209
Iteration    4 => Loss: 0.26488539455552501112
Iteration    5 => Loss: 0.21974966586468008822
Iteration    6 => Loss: 0.21169719667449560241
Iteration    7 => Loss: 0.20475082478710038281
Iteration    8 => Loss: 0.19886878484015674262
Iteration    9 => Loss: 0.19372162858641284178
[5] Success 9268/10000: (92.68)


Bonus: Run if for all numbers

In [34]:
def encode_number(Y, number):
  encoded_Y = np.zeros_like(Y)
  n_labels = Y.shape[0]
  for i in range(n_labels):
    if Y[i] == number: 
      encoded_Y[i][0] = 1
  return encoded_Y

for i in range(1, 10):
  Y_train = encode_number(Y_label, i)
  Y_test = encode_number(Y_test_label, i)
  
  w = train(X_train, Y_train, iterations=10, lr=1e-5)
  test(X_test, Y_test, w, i)
  

Iteration    0 => Loss: 0.69314718055994528623
Iteration    1 => Loss: 0.63297126749254217781
Iteration    2 => Loss: 0.40415041206134089125
Iteration    3 => Loss: 0.20740137552095117002
Iteration    4 => Loss: 0.10475739758236242427
Iteration    5 => Loss: 0.08406584425996900534
Iteration    6 => Loss: 0.08017763379914089050
Iteration    7 => Loss: 0.07779278786107958477
Iteration    8 => Loss: 0.07571922657128110634
Iteration    9 => Loss: 0.07385842309425814933
Correct Guesses for 1: 9837
Success Precentage:
98.37
Iteration    0 => Loss: 18.31350302813820007941
Iteration    1 => Loss: 1.35295391222630434491
Iteration    2 => Loss: 28.14728995527894639395
Iteration    3 => Loss: 49.86778039870669942957
Iteration    4 => Loss: 35.83839687987449451612
Iteration    5 => Loss: 81.35740667903965572805
Iteration    6 => Loss: 38.47093873551957443624
Iteration    7 => Loss: 103.03195754120535809761
Iteration    8 => Loss: 57.36743613602427416254
Iteration    9 => Loss: 113.7423947110573152

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


Iteration    5 => Loss: inf
Iteration    6 => Loss: inf
Iteration    7 => Loss: inf
Iteration    8 => Loss: inf
Iteration    9 => Loss: inf
Correct Guesses for 6: 4031
Success Precentage:
40.31
Iteration    0 => Loss: 19.22143341504765245986
Iteration    1 => Loss: 226.00351521287157652296
Iteration    2 => Loss: 230.18426211367739142588
Iteration    3 => Loss: inf
Iteration    4 => Loss: inf
Iteration    5 => Loss: inf
Iteration    6 => Loss: inf
Iteration    7 => Loss: inf
Iteration    8 => Loss: inf
Iteration    9 => Loss: inf
Correct Guesses for 7: 3704
Success Precentage:
37.04
Iteration    0 => Loss: 17.99705823530656445541
Iteration    1 => Loss: 355.25793636014117282684


KeyboardInterrupt: ignored